In [ ]:
import numpy as np
from scipy.integrate import quad 
import matplotlib.pyplot as plt

In [ ]:
#Calculates the decoherence function as a function of time, wc is the value of the cut-off frequency ($\omega_c$), 
#s is the ohmicity parameter, n is the coupling strength ($\eta$) and t is the time.
def decoherence(wc, s, n, t):
    
    #Defining the function to be integrated
    def f(w, wc, s, n, t):
        sd = n*wc**(1-s)*w**s*np.exp(-w/wc)
        f = sd*(1-np.cos(w*t))/w**2
        return(f)
    
    #Numerical Integration
    dec, err = quad(f, 0, np.inf, args = (wc, s, n, t), limit=400) 
    dec *= 4
    
    return(dec)

In [ ]:
#Calculates the density matrix at time t. wc is the value of the cut-off frequency ($\omega_c$), s is the ohmicity 
#parameter, n is the coupling strength ($\eta$) and rho0 is the density matrix at time t=0.
def rhot(t, wc, s, n, rho0):
    
    #calculating the value of the decoherence function at time t
    dec = decoherence(wc, s, n, t)
    
    #calculating the density matrix at time t
    rhot = np.array([[rho0[0,0], rho0[0, 1]*np.exp(-dec)],[rho0[1,0]*np.exp(-dec), rho0[1,1]]])
    return(rhot)

In [ ]:
#defining the Pauli matrices
sx = np.array([[0, 1],[1, 0]]) #$\sigma_x$
sy = np.array([[0, -1j],[1j, 0]]) #$\sigma_y$
sz = np.array([[1, 0],[0, -1]]) #$\sigma_z$

In [ ]:
#this function generates a training set with n_training evolutions, along with the corresponding labels. n_training
#is the number of training examples to be generated, which should be a multiple of 3. a is the initial time for the 
#trajectories, b is the final time and dt is the time step. n_l and wc_l are the minimum values of $\eta$ and 
#$\omega_c$ while n_h and wc_h are the maximum values. Likewise, super_l and sub_l are the minimum values of s for 
#the sub-Ohmic and super-Ohmic spectral densities, while super_h and sub_h are the maximum values. rho0 is the
#density matrix at time t=0.
def generate_data(n_training, a, b, dt, n_l, n_h, wc_l, wc_h, super_l, super_h, sub_l, sub_h, rho0):
    
    #generating an array of times 
    t = np.arange(a, b+dt, dt)
    
    #defining arrays to store the trajectories. xtrainx stores the expectation value of $\sigma_x$, xtrainy stores
    #the expectation value of $\sigma_y$ and xtrainz stores the expectation value of $\sigma_z$.
    xtrainx = np.zeros((n_training, len(t)), dtype=complex)
    xtrainy = np.zeros((n_training, len(t)), dtype=complex)
    xtrainz = np.zeros((n_training, len(t)), dtype=complex)
    
    #defining an array to store the labels and the values of the parameter.y_train is an array of shape
    # n_training x 6. if the ith training example is sub-Ohmic then element [i,0] will be set equal to 1, if it is
    # Ohmic then [i,1] will be set equal to 1 and if it is super-Ohmic then [i,2] will be set equal to 1. 
    # element [i,3] is set equal to $\omega_c$ while elements [i,4] and [i,5] are set equal to s and $\eta$ 
    #respectively. The values of $\omega_c$, s and $\eta$ will not be passed to the neural network. 
    ytrain = np.zeros((n_training,6)) 
    
    for i in range(int(n_training/3)):
        #sets the first third of the training examples equal to evolutions with a sub-Ohmic spectral density 
        wc = np.random.uniform(wc_l, wc_h)
        s = np.random.uniform(sub_l, sub_h)
        n = np.random.uniform(n_l,n_h)
        
        for j in range(len(t)):
            rho = rhot(t[j], wc, s, n, rho0)
            xtrainx[i,j] = np.trace(sx@rho)
            xtrainy[i,j] = np.trace(sy@rho)
            xtrainz[i,j] = np.trace(sz@rho)
        
        ytrain[i,0]=1
        ytrain[i,3]=wc
        ytrain[i,4]=s
        ytrain[i,5]=n
        
        #sets the second third of the training examples equal to evolutions with an Ohmic spectral density
        wc = np.random.uniform(wc_l, wc_h)
        s = 1
        n = np.random.uniform(n_l,n_h)
        
        for j in range(len(t)):
            rho = rhot(t[j], wc, s, n, rho0)
            xtrainx[i + int(n_training/3),j] = np.trace(sx@rho) 
            xtrainy[i + int(n_training/3),j] = np.trace(sy@rho)
            xtrainz[i + int(n_training/3),j] = np.trace(sz@rho)
        
        ytrain[i+int(n_training/3),1]=1
        ytrain[i+int(n_training/3),3]=wc
        ytrain[i+int(n_training/3),4]=s
        ytrain[i+int(n_training/3),5]=n
        
        #sets the last third of the training examples equal to evolutions with a super-Ohmic spectral density
        wc = np.random.uniform(wc_l, wc_h)
        s = np.random.uniform(super_l, super_h)
        n = np.random.uniform(n_l,n_h)
        
        for j in range(len(t)):
            rho = rhot(t[j], wc, s, n, rho0)
            xtrainx[i + 2*int(n_training/3),j] = np.trace(sx@rho)
            xtrainy[i + 2*int(n_training/3),j] = np.trace(sy@rho)
            xtrainz[i + 2*int(n_training/3),j] = np.trace(sz@rho)
        
        ytrain[i+2*int(n_training/3),2]=1
        ytrain[i+2*int(n_training/3),3]=wc
        ytrain[i+2*int(n_training/3),4]=s
        ytrain[i+2*int(n_training/3),5]=n
    
    #shuffling the training set
    indices_training = np.random.permutation(n_training)
    Xtrainx = xtrainx[indices_training]
    Xtrainy = xtrainy[indices_training]
    Xtrainz = xtrainz[indices_training]
    Ytrain = ytrain[indices_training]
        
    return(Xtrainx, Xtrainy, Xtrainz, Ytrain)